In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import math

## Configurações iniciais

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(1800, 1800, "Space", None, None)
glfw.make_context_current(window)

In [3]:
vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,1.0);
        }
        """

In [4]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

In [5]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


In [6]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

In [7]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

In [8]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

In [9]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


In [10]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

## Definição dos objetos
Abaixo é definido uma lista de objetos que estarão presentes na cena do projeto. Cada objeto é representado por um dicionário contendo informações sobre suas características e como deve ser desenhado. A lista objects inclui os seguintes atributos para cada objeto:

- name: Nome do objeto, utilizado para identificação.
- transformation: Dicionário que especifica as transformações aplicáveis ao objeto:
    - translation: Vetor de translação (deslocamento) no espaço 3D.
    - rotation: Vetor de rotação, especificado pelos ângulos de rotação ao longo dos eixos X, Y e Z.
    - scale: Fatores de escala ao longo dos eixos X, Y e Z.
    - object_position: Posição inicial do objeto na cena.
- draw: Função que desenha o objeto. 

In [11]:
objects = [
    {
        'name': 'rocket',
        'transformation': {
            'translation': (0.0, 0.0, 0.0),
            'rotation': (0.0, 0.0, 0.0),
            'scale': (1.0, 1.0, 1.0),
            'object_position': (-0.7, -0.3, 0.0)
        },
        'draw': lambda: draw_rocket()
    },
    {
        'name': 'saturn',
        'transformation': {
            'translation': (0.0, 0.0, 0.0),
            'rotation': (2*math.pi/3, 0.0, 0.0),
            'scale': (1.0, 1.0, 1.0),
            'object_position': (0.5, 0.5, 0.0)
        },
        'draw': lambda: draw_saturn()
    },
    {
        'name': 'satellite',
        'transformation': {
            'translation': (0.0, 0.0, 0.0),
            'rotation': (-math.pi/4, 3*math.pi/4, 0.0),
            'scale': (0.5, 0.5, 0.5),
            'object_position': (0.5, -0.5, 0.0)
        },
        'draw': lambda: draw_satellite()
    }, 
    {
        'name': 'star',
        'transformation': {
            'translation': (0.0, 0.0, 0.0),
            'rotation': (0.0, 0.0, 0.0),
            'scale': (0.6, 0.6, 0.6),
            'object_position': (-0.2, 0.5, 0.0)
        },
        'draw': lambda: draw_star()
    }, 
        {
        'name': 'moon',
        'transformation': {
            'translation': (0.0, 0.0, 0.0),
            'rotation': (0.0, 0.0, 0.0),
            'scale': (0.5, 0.5, 0.5),
            'object_position': (-0.7, 0.6, 0.0)
        },
        'draw': lambda: draw_moon()
    }
]

# Index of the currently selected object
current_object_index = 0 

## Geração dos vértices dos objetos
Para gerar os vértices dos objetos definidos no dicionário, foram usadas funções para criar os componentes individuais de cada objeto e, em seguida, compor esses componentes para formar o objeto completo.

In [12]:
# Generate the cone vertices with a given radius and height
def generate_cone_vertices(r=0.1, H=0.1, position=(0.0, 0.0, 0.0)):
    num_sectors = 20  # Number of sectors (longitude)
    
    # Angle step for the sectors (longitude)
    sector_step = (2 * math.pi) / num_sectors  # Varies from 0 to 2π
    
    # Helper function to generate vertex coordinates on the base circle
    def coord_cone(t, r):
        x = r * math.cos(t)
        y = r * math.sin(t)
        return (x, y)
    
    # Generate vertices for the base of the cone
    base_vertices = []
    for i in range(num_sectors):
        angle = i * sector_step
        x, y = coord_cone(angle, r)
        base_vertices.append((position[0], y + position[1], x + position[2]))  # Base z = position[2]

    # Add the apex of the cone
    apex = (H + position[0], position[1], position[2])
    
    vertices_list = []
    
    # Create triangles for the base
    for i in range(num_sectors):
        p0 = base_vertices[i]
        p1 = base_vertices[(i + 1) % num_sectors]
        vertices_list.append(p0)
        vertices_list.append(p1)
        vertices_list.append((position[0], position[1], position[2]))  # The center of the base
    
    # Create triangles for the sides of the cone
    for i in range(num_sectors):
        base_vertex = base_vertices[i]
        next_base_vertex = base_vertices[(i + 1) % num_sectors]
        vertices_list.append(base_vertex)
        vertices_list.append(next_base_vertex)
        vertices_list.append(apex)
    
    total_vertices = len(vertices_list)
    vertices = np.zeros(total_vertices, [("position", np.float32, 3)])
    vertices['position'] = np.array(vertices_list)
    
    return vertices['position']

# Generate the cylinder vertices with a given radius and height
def generate_cylinder_vertices(r=0.1, H=0.3, position=(0.0, 0.0, 0.0)):
    num_sectors = 20 # Number of sectors (longitude)
    num_stacks = 20 # Number of stacks (Latitude)
    
    # Grid sectors vs stacks (longitude vs latitude)
    sector_step = (math.pi * 2) / num_sectors # Varies from 0 to 2π
    stack_step = H / num_stacks # Varies from 0 to H
    
    # Helper function to generate vertex coordinates of the cylinder
    def coord_cylinder(t, h, r):
        x = r * math.cos(t)
        y = r * math.sin(t)
        z = h
        return (z + position[0], y + position[1], x + position[2])
    
    vertices_list = []
    for j in range(0, num_stacks): # For each stack (latitude)
        
        for i in range(0, num_sectors): # For each sector (longitude) 
            
            u = i * sector_step # Sector angle
            v = j * stack_step # Stack height
            
            un = 0 # Next sector angle
            if i + 1 == num_sectors:
                un = math.pi * 2
            else: un = (i + 1) * sector_step
                
            vn = 0 # Next stack height
            if j + 1 == num_stacks:
                vn = H
            else: vn = (j + 1) * stack_step
            
            # Polygon vertices
            p0 = coord_cylinder(u, v, r)
            p1 = coord_cylinder(u, vn, r)
            p2 = coord_cylinder(un, v, r)
            p3 = coord_cylinder(un, vn, r)
            
            # Triangle 1 (first part of the polygon)
            vertices_list.append(p0)
            vertices_list.append(p2)
            vertices_list.append(p1)
            
            # Triangle 2 (second part of the polygon)
            vertices_list.append(p3)
            vertices_list.append(p1)
            vertices_list.append(p2)
                        
            if v == 0:
                vertices_list.append(p0)
                vertices_list.append(p2)
                vertices_list.append(coord_cylinder(0, v, 0))
            if vn == H:
                # Make a triangle from the same angle u, but with heights h = vn 
                vertices_list.append(p1)
                vertices_list.append(p3)
                vertices_list.append(coord_cylinder(0, vn, 0))
    
    total_vertices = len(vertices_list)
    vertices = np.zeros(total_vertices, [("position", np.float32, 3)])
    vertices['position'] = np.array(vertices_list)

    return vertices['position']

# Generate the prism vertices with a given width, height, and depth
def generate_prism_vertices(width=0.2, H=0.2, depth=0.1, position=(0.0, 0.0, 0.0)):
    w, h, d = width / 2, H / 2, depth / 10  
    
    vertices_list = [
        # Back Triangle Face
        (position[0], position[1], position[2] - d),
        (position[0], position[1] + h, position[2] - d),
        (position[0] + w, position[1], position[2] - d),

        # Front Triangle Face
        (position[0], position[1], position[2] + d),
        (position[0], position[1] + h, position[2] + d),
        (position[0] + w, position[1], position[2] + d),

        # Left Rectangle Face
        (position[0], position[1], position[2] - d),
        (position[0], position[1], position[2] + d),
        (position[0], position[1] + h, position[2] - d),
        (position[0], position[1] + h, position[2] + d),

        # Top Rectangle Face
        (position[0], position[1] + h, position[2] - d),
        (position[0], position[1] + h, position[2] + d),
        (position[0] + w, position[1], position[2] - d),
        (position[0] + w, position[1], position[2] + d),
    ]
    
    total_vertices = len(vertices_list)
    vertices = np.zeros(total_vertices, [("position", np.float32, 3)])
    vertices['position'] = np.array(vertices_list)

    return vertices['position']

# Using the components cone, cylinder and prism, create the rocket vertices
def generate_rocket_vertices(start_position=(0.0, 0.0, 0.0)):
    # List to store the number of vertices of each component used in the object
    num_vertices_components = []
    
    x_0 = start_position[0]
    y_0 = start_position[1]
    z_0 = start_position[2]

    # Define the start position of each component
    cone_position = (x_0, y_0, z_0)
    cylinder_position = (x_0 - 0.15, y_0, z_0)
    prism1_position = (x_0 - 0.15, y_0 + 0.1, z_0)
    prism2_position = (x_0 - 0.15, y_0 - 0.1, z_0)

    # Generate the vertices of the components, with the desired parameters
    cone_vertices = generate_cone_vertices(H=0.1, position=cone_position)
    num_vertices_components.append(len(cone_vertices))
    
    cylinder_vertices = generate_cylinder_vertices(H=0.15, position=cylinder_position)
    num_vertices_components.append(len(cylinder_vertices))
    
    prism1_vertices = generate_prism_vertices(position=prism1_position)
    num_vertices_components.append(len(prism1_vertices))

    prism2_vertices = generate_prism_vertices(H=-0.2, position=prism2_position)
    num_vertices_components.append(len(prism2_vertices))

    # Combine all vertices
    vertices = np.vstack((cone_vertices, cylinder_vertices, prism1_vertices, prism2_vertices))
    
    return vertices, num_vertices_components

# Generate the sphere vertices with a given radius
def generate_sphere_vertices(r=0.25, position=(0.0, 0.0, 0.0)):
    num_sectors = 32  # Number of sectors (longitude)
    num_stacks = 32  # Number of stacks (Latitude)
    
    sector_step = (math.pi * 2) / num_sectors 
    stack_step = (math.pi) / num_stacks
    
    # Helper function to generate vertex coordinates of the sphere
    def F(u,v,r):
        x = r*math.sin(v)*math.cos(u)
        y = r*math.sin(v)*math.sin(u)
        z = r*math.cos(v)
        return (x + position[0], y + position[1], z + position[2])

    vertices_list = []
    for i in range(0, num_sectors): 
        for j in range(0, num_stacks): 
            u = i * sector_step # sector angle
            v = j * stack_step # stack angle
            
            un = 0 # Next sector angle
            if i + 1 == num_sectors:
                un = math.pi * 2
            else: un = (i + 1) * sector_step
                
            vn = 0 # Next stack angle
            if j + 1 == num_stacks:
                vn = math.pi
            else: vn = (j + 1) * stack_step
            
            # polygon vertices
            p0 = F(u, v, r)
            p1 = F(u, vn, r)
            p2 = F(un, v, r)
            p3 = F(un, vn, r)
            
            # Triangle 1
            vertices_list.append(p0)
            vertices_list.append(p2)
            vertices_list.append(p1)
            
            # Triangle 2
            vertices_list.append(p3)
            vertices_list.append(p1)
            vertices_list.append(p2)
    
    
    total_vertices = len(vertices_list)
    vertices = np.zeros(total_vertices, [("position", np.float32, 3)])
    vertices['position'] = np.array(vertices_list)
    
    return vertices['position']

# Function to generate the vertices of a ring (annulus) with a given inner and outer radius
def generate_ring_vertices(inner_radius=0.30, outer_radius=0.40, position=(0.0, 0.0, 0.0)):
    ring_vertices_list = []
    
    num_sectors = 32  # Number of sectors (angular divisions)
    num_rings = 8  # Number of radial divisions between the inner and outer radius
    
    # Step for each sector (angle step)
    sector_step = (math.pi * 2) / num_sectors  # Sector step from 0 to 2π
   
    # Step for each ring (distance between inner and outer radius)
    ring_step = (outer_radius - inner_radius) / num_rings  # Ring step size
    
    for i in range(num_sectors):
        for j in range(num_rings):
            u = i * sector_step  # Current sector angle
            r1 = inner_radius + j * ring_step  # Current radius at the inner edge of the ring
            r2 = r1 + ring_step  # Next radius at the outer edge of the ring
            
            # Next sector angle, wrapping around to 2π if at the last sector
            u_next = (i + 1) * sector_step if i + 1 < num_sectors else math.pi * 2
            
            # Vertices for the current quadrilateral in the ring:
            # p0 and p1 are on the inner edge, p2 and p3 are on the outer edge
            p0 = (r1 * math.cos(u) + position[0], r1 * math.sin(u) + position[1], position[2])
            p1 = (r1 * math.cos(u_next) + position[0], r1 * math.sin(u_next) + position[1], position[2])
            p2 = (r2 * math.cos(u) + position[0], r2 * math.sin(u) + position[1], position[2])
            p3 = (r2 * math.cos(u_next) + position[0], r2 * math.sin(u_next) + position[1], position[2])
            
            # Create two triangles for the current quadrilateral
            ring_vertices_list.extend([p0, p2, p1])  # First triangle
            ring_vertices_list.extend([p1, p2, p3])  # Second triangle
    
    total_ring_vertices = len(ring_vertices_list)
    ring_vertices = np.zeros(total_ring_vertices, [("position", np.float32, 3)])
    ring_vertices['position'] = np.array(ring_vertices_list)  # Store the vertex positions

    return ring_vertices['position'] 

# Using the components sphere, and ring, create the planet Saturn vertices
def generate_saturn_vertices(start_position=(0.0, 0.0, 0.0)):
    num_vertices_components = []

    x_0 = start_position[0]
    y_0 = start_position[1]
    z_0 = start_position[2]

    # Define the start position of each component
    sphere_position = (x_0, y_0, z_0)
    ring_position = (x_0, y_0, z_0)

    # Generate the vertices of the components
    sphere_vertices = generate_sphere_vertices(position=sphere_position)
    num_vertices_components.append(len(sphere_vertices))

    ring_vertices = generate_ring_vertices(position=ring_position)
    num_vertices_components.append(len(ring_vertices))

    vertices = np.vstack((sphere_vertices, ring_vertices))
    
    return vertices, num_vertices_components

# Generate the parallelepiped vertices with given width, height and depth
def generate_parallelepiped_vertices(width=0.4, height=0.2, depth=0.1, position=(0.0, 0.0, 0.0)):
    w, h, d = width / 2, height / 2, depth / 2
    
    vertices_list = [
        # Front face
        (position[0] - w, position[1] - h, position[2] - d),
        (position[0] + w, position[1] - h, position[2] - d),
        (position[0] + w, position[1] + h, position[2] - d),
        (position[0] - w, position[1] + h, position[2] - d),
        
        # Back face
        (position[0] - w, position[1] - h, position[2] + d),
        (position[0] + w, position[1] - h, position[2] + d),
        (position[0] + w, position[1] + h, position[2] + d),
        (position[0] - w, position[1] + h, position[2] + d),
        
        # Bottom face
        (position[0] - w, position[1] - h, position[2] - d),
        (position[0] + w, position[1] - h, position[2] - d),
        (position[0] + w, position[1] - h, position[2] + d),
        (position[0] - w, position[1] - h, position[2] + d),
        
        # Top face
        (position[0] - w, position[1] + h, position[2] - d),
        (position[0] + w, position[1] + h, position[2] - d),
        (position[0] + w, position[1] + h, position[2] + d),
        (position[0] - w, position[1] + h, position[2] + d),
        
        # Left face
        (position[0] - w, position[1] - h, position[2] - d),
        (position[0] - w, position[1] + h, position[2] - d),
        (position[0] - w, position[1] + h, position[2] + d),
        (position[0] - w, position[1] - h, position[2] + d),
        
        # Right face
        (position[0] + w, position[1] - h, position[2] - d),
        (position[0] + w, position[1] + h, position[2] - d),
        (position[0] + w, position[1] + h, position[2] + d),
        (position[0] + w, position[1] - h, position[2] + d)
    ]
    
    total_vertices = len(vertices_list)
    vertices = np.zeros(total_vertices, [("position", np.float32, 3)])
    vertices['position'] = np.array(vertices_list)

    return vertices['position']

# Using the components cylinder, cone, and parallelepiped components
# create the satellite vertices
def generate_satellite_vertices(start_position=(0.0, 0.0, 0.0)):
    num_vertices_components = []

    # Define the start position of the satellite
    x_0 = start_position[0]
    y_0 = start_position[1]
    z_0 = start_position[2]

    # Define the start position of each component
    cylinder1_position = (x_0, y_0, z_0)
    cone1_position = (x_0 + 0.50, y_0, z_0)
    cone2_position = (x_0 + 0.50, y_0, z_0)
    parallelepiped1_position = (x_0 + 0.20, y_0 + 0.50, z_0)
    parallelepiped2_position = (x_0 + 0.20, y_0 - 0.50, z_0)
    parallelepiped3_position = (x_0 + 0.20, y_0 + 0.20, z_0)
    parallelepiped4_position = (x_0 + 0.20, y_0 - 0.20, z_0)

    # Generate the components vertices
    cylinder1_vertices = generate_cylinder_vertices(r=0.15, H=0.4, position=cylinder1_position)
    num_vertices_components.append(len(cylinder1_vertices))

    cone1_vertices = generate_cone_vertices(r=0.2, H=-0.1, position=cone1_position)
    num_vertices_components.append(len(cone1_vertices))

    cone2_vertices = generate_cone_vertices(r=0.05, H=0.08, position=cone2_position)
    num_vertices_components.append(len(cone2_vertices))

    p1 = generate_parallelepiped_vertices(width=0.15, height=0.6, depth=0.05, position=parallelepiped1_position)
    num_vertices_components.append(len(p1))

    p2 = generate_parallelepiped_vertices(width=0.15, height=0.6, depth=0.05, position=parallelepiped2_position)
    num_vertices_components.append(len(p2))

    p3 = generate_parallelepiped_vertices(width=0.05, height=0.1, depth=0.025, position=parallelepiped3_position)
    num_vertices_components.append(len(p3))
    
    p4 = generate_parallelepiped_vertices(width=0.05, height=0.1, depth=0.025, position=parallelepiped4_position)
    num_vertices_components.append(len(p4))

    vertices = np.vstack((cylinder1_vertices, cone1_vertices, cone2_vertices, p1, p2, p3, p4))
    
    return vertices, num_vertices_components

# Generate the vertices of the star
def generate_star_vertices(start_position=(0.0, 0.0, 0.0)):
    num_vertices_comp = []

    # Define the start position
    x_0, y_0, z_0 = start_position
    
    vertices_list = [
        # Triangle 1
        (x_0 - 0.130, y_0 + 0.0, z_0 + 0.0),
        (x_0 + 0.0, y_0 -0.232,  z_0 + 0.0),
        (x_0 + 0.130, y_0 + 0.0,  z_0 + 0.0),

        # Triangle 2
        (x_0 - 0.130, y_0 - 0.153,  z_0 + 0.0),
        (x_0 + 0.0, y_0 + 0.082,  z_0 + 0.0),
        (x_0 + 0.130, y_0 - 0.153,  z_0 + 0.0)
    ]
    
    total_vertices = len(vertices_list)
    num_vertices_comp.append(total_vertices)
    vertices = np.zeros(total_vertices, [("position", np.float32, 3)])
    vertices['position'] = np.array(vertices_list)

    return vertices['position'], num_vertices_comp

# Generate the crescent moon vertices, with given radius of the outer and inner circle
def generate_moon_vertices(radius=0.35, shadow_radius=0.25, position=(0.0, 0.0, 0.0)):
    num_vertices_comp = []
    num_segments = 30

    # Helper function to generate vertex coordinates of the circle
    def circle_points(radius, center, num_segments):
        angles = np.linspace(0, 2 * math.pi, num_segments, endpoint=False)
        points = [(radius * math.cos(angle) + center[0], radius * math.sin(angle) + center[1], center[2]) for angle in angles]
        return points
    
    # Helper function to find the intersection of 2 circles
    def find_circle_intersections(x1, y1, R1, x2, y2, R2):
        # Distance between the centers
        d = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
        
        # Check for no intersection
        if d > R1 + R2:
            return None, None
        
        # Check if one circle is inside the other
        if d < abs(R1 - R2):
            return None, None
        
        # Check for coincident circles
        if d == 0 and R1 == R2:
            return None, None
        
        # Calculate the point of intersection of the line joining the circle centers
        a = (R1 ** 2 - R2 ** 2 + d ** 2) / (2 * d)
        h = math.sqrt(R1 ** 2 - a ** 2)
        
        # Midpoint coordinates
        Px = x1 + a * (x2 - x1) / d
        Py = y1 + a * (y2 - y1) / d
        
        # Intersection points
        intersection1 = (Px + h * (y2 - y1) / d, Py - h * (x2 - x1) / d)
        intersection2 = (Px - h * (y2 - y1) / d, Py + h * (x2 - x1) / d)
        
        return intersection1, intersection2
    
    # Generate vertices for the larger circle (full moon)
    outer_circle = circle_points(radius, position, num_segments)
    
    # Generate vertices for the smaller circle (shadow)
    shadow_circle = circle_points(shadow_radius, (position[0] + radius * 0.35, position[1], position[2]), num_segments)

    # Find the intersection points of the circles
    intersection1, intersection2 = find_circle_intersections(position[0], position[1], radius, position[0] + radius * 0.5, position[1], shadow_radius)
    intersection_x = min(intersection1[0], intersection2[0]) + 0.06 # Use the smaller x of the two intersections
    
    # Combine circles and create triangles for the crescent shape
    vertices_list = []
    for i in range(num_segments):
        p0 = outer_circle[i]
        p1 = outer_circle[(i + 1) % num_segments]
        p2 = shadow_circle[i]

        p3 = shadow_circle[i]
        p4 = outer_circle[(i + 1) % num_segments]
        p5 = shadow_circle[(i + 1) % num_segments]

        if p0[0] <= intersection_x:
            vertices_list.extend([p0, p1, p2])
        if p3[0] <= intersection_x :
            vertices_list.extend([p3, p4, p5])
    
    # Create numpy array for vertex data
    total_vertices = len(vertices_list)
    num_vertices_comp.append(total_vertices)
    vertices = np.zeros(total_vertices, [("position", np.float32, 3)])
    vertices['position'] = np.array(vertices_list)
    
    return vertices['position'], num_vertices_comp

# Generate the vertices of all objects, given the start position of each one
spaceship_vertices, num_vertices_spaceship = generate_rocket_vertices(start_position=(-0.7, -0.3, 0.0))
saturn_vertices, num_vertices_saturn = generate_saturn_vertices(start_position=(0.5, 0.5, 0.0))
satellite_vertices, num_vertices_satellite = generate_satellite_vertices(start_position=(0.5, -0.5, 0.0))
star_vertices, num_vertices_star = generate_star_vertices(start_position=(-0.2, 0.5, 0.0))
moon_vertices, num_vertices_moon = generate_moon_vertices(position=(-0.7, 0.6, 0.0))

# Save the vertices of all objects
vertices = np.vstack((spaceship_vertices, saturn_vertices, satellite_vertices, star_vertices, moon_vertices))

# Save the number of vertices of each object 
num_vertices_comp = np.concatenate((num_vertices_spaceship, num_vertices_saturn, num_vertices_satellite, num_vertices_star, num_vertices_moon), axis=0)

print(num_vertices_comp)

[ 120 2520   14   14 6144 1536 2520  120  120   24   24   24   24    6
  144]


In [13]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

In [14]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

In [15]:
# Bind the position attribute
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


In [16]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

In [17]:
glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

In [18]:
loc_color = glGetUniformLocation(program, "color")

In [19]:
glfw.show_window(window)

In [20]:
# Function to activate or deactivate the polygon mode of the objects
def toggle_polygon_mode(flag_polygon_mode):
    if flag_polygon_mode == True:
        glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)
    elif flag_polygon_mode == False:
        glPolygonMode(GL_FRONT_AND_BACK, GL_FILL)   

In [21]:
# Flag that indicates if the polygon mode is activated or deactivated
flag_polygon_mode = True

# Function that handle the keyboard events
def key_event(window, key, scancode, action, mods):
    global objects, current_object_index, flag_polygon_mode, selected_object_text

    # Get the current object selected
    obj = objects[current_object_index]

    # x, y, z increments for translation
    x_inc = 0.03
    y_inc = 0.03
    z_inc = 0.03

    # x, y, z increments for rotation
    rx_inc = 0.1
    ry_inc = 0.1
    rz_inc = 0.1
    
    # scale increment 
    s_inc = 0.05

    # Translation
    if key == 263: # Left arrow key
        obj['transformation']['translation'] = (obj['transformation']['translation'][0] - x_inc, obj['transformation']['translation'][1], obj['transformation']['translation'][2])
    if key == 262: # Right arrow key
        obj['transformation']['translation'] = (obj['transformation']['translation'][0] + x_inc, obj['transformation']['translation'][1], obj['transformation']['translation'][2])
    if key == 265: # Up arrow key
        obj['transformation']['translation'] = (obj['transformation']['translation'][0], obj['transformation']['translation'][1] + y_inc, obj['transformation']['translation'][2])
    if key == 264: # Down arrow key
        obj['transformation']['translation'] = (obj['transformation']['translation'][0], obj['transformation']['translation'][1] - y_inc, obj['transformation']['translation'][2])
    if key == 77: # M key
        obj['transformation']['translation'] = (obj['transformation']['translation'][0], obj['transformation']['translation'][1], obj['transformation']['translation'][2] + z_inc)
    if key == 78: # N key
        obj['transformation']['translation'] = (obj['transformation']['translation'][0], obj['transformation']['translation'][1], obj['transformation']['translation'][2] - z_inc)
   
    # Rotation
    if key == 65: # A key
        obj['transformation']['rotation'] = (obj['transformation']['rotation'][0] - rx_inc, obj['transformation']['rotation'][1], obj['transformation']['rotation'][2])
    if key == 83: # S key
        obj['transformation']['rotation'] = (obj['transformation']['rotation'][0] + rx_inc, obj['transformation']['rotation'][1], obj['transformation']['rotation'][2])
    if key == 68: # D key
        obj['transformation']['rotation'] = (obj['transformation']['rotation'][0], obj['transformation']['rotation'][1]  - ry_inc, obj['transformation']['rotation'][2])
    if key == 70: # F key
        obj['transformation']['rotation'] = (obj['transformation']['rotation'][0], obj['transformation']['rotation'][1] + ry_inc, obj['transformation']['rotation'][2])
    if key == 71: # G key
        obj['transformation']['rotation'] = (obj['transformation']['rotation'][0], obj['transformation']['rotation'][1], obj['transformation']['rotation'][2] + rz_inc)
    if key == 72: # H key
        obj['transformation']['rotation'] = (obj['transformation']['rotation'][0], obj['transformation']['rotation'][1], obj['transformation']['rotation'][2] - rz_inc)

    # Scale
    if key == 90: # Z key
        obj['transformation']['scale'] = (obj['transformation']['scale'][0] + s_inc, obj['transformation']['scale'][1] + s_inc, obj['transformation']['scale'][2] + s_inc)
    if key == 88 : # X key
        obj['transformation']['scale'] = (obj['transformation']['scale'][0] - s_inc, obj['transformation']['scale'][1] - s_inc, obj['transformation']['scale'][2] - s_inc)

    # Toggle polygon mode
    if key == glfw.KEY_P and action == glfw.PRESS:
        flag_polygon_mode = not flag_polygon_mode  # Toggle the flag
        toggle_polygon_mode(flag_polygon_mode) 

    # Change current selected object
    if key == glfw.KEY_TAB and action == glfw.PRESS:
        current_object_index = (current_object_index + 1) % len(objects)
    print(key)

    
glfw.set_key_callback(window, key_event)

In [22]:
# Function that multiply two matrices
def mult_matrix(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

# Function that applies the desired transformation of the chosen object
def apply_transformations(matrix, translation=(0, 0, 0), rotation=(0, 0, 0), scale=(1, 1, 1), object_position=(0, 0, 0)):  
    # Object's position
    obj_x, obj_y, obj_z = object_position
    
    # Translate the object to the origin
    mat_translate_to_origin = np.array([ 1.0,  0.0, 0.0, -obj_x, 
                                         0.0,  1.0, 0.0, -obj_y, 
                                         0.0,  0.0, 1.0, -obj_z, 
                                         0.0,  0.0, 0.0, 1.0], np.float32)

    # Translation matrix (move object to the final position)
    t_x, t_y, t_z = translation
    mat_translate = np.array([     1.0,  0.0, 0.0, t_x, 
                                   0.0,  1.0, 0.0, t_y, 
                                   0.0,  0.0, 1.0, t_z, 
                                   0.0,  0.0, 0.0, 1.0], np.float32)

    # Rotation matrices (applied in x, y, z order)
    r_x, r_y, r_z = rotation
    cos_rx, sin_rx = math.cos(r_x), math.sin(r_x)
    cos_ry, sin_ry = math.cos(r_y), math.sin(r_y)
    cos_rz, sin_rz = math.cos(r_z), math.sin(r_z)

    mat_rotation_x = np.array([    1.0,   0.0,    0.0, 0.0, 
                                   0.0, cos_rx, -sin_rx, 0.0, 
                                   0.0, sin_rx,  cos_rx, 0.0, 
                                   0.0,   0.0,    0.0, 1.0], np.float32)
    
    mat_rotation_y = np.array([   cos_ry,  0.0, sin_ry, 0.0, 
                                   0.0,    1.0,   0.0, 0.0, 
                                  -sin_ry, 0.0, cos_ry, 0.0, 
                                   0.0,    0.0,   0.0, 1.0], np.float32)
    
    mat_rotation_z = np.array([   cos_rz, -sin_rz, 0.0, 0.0, 
                                  sin_rz,  cos_rz, 0.0, 0.0, 
                                   0.0,      0.0, 1.0, 0.0, 
                                   0.0,      0.0, 0.0, 1.0], np.float32)

    # Scaling matrix
    s_x, s_y, s_z = scale
    mat_scale = np.array([        s_x,  0.0, 0.0, 0.0, 
                                  0.0,  s_y, 0.0, 0.0, 
                                  0.0,  0.0, s_z, 0.0, 
                                  0.0,  0.0, 0.0, 1.0], np.float32)

    # Translate object to origin before scaling and rotation
    matrix = mult_matrix(mat_translate_to_origin, matrix)
    
    # Apply transformations: Scale -> Rotate
    matrix = mult_matrix(mat_scale, matrix)
    matrix = mult_matrix(mat_rotation_x, matrix)
    matrix = mult_matrix(mat_rotation_y, matrix)
    matrix = mult_matrix(mat_rotation_z, matrix)

    # Translate object back to its original position
    mat_translate_back = np.array([  1.0,  0.0, 0.0, obj_x, 
                                     0.0,  1.0, 0.0, obj_y, 
                                     0.0,  0.0, 1.0, obj_z, 
                                     0.0,  0.0, 0.0, 1.0], np.float32)
    
    matrix = mult_matrix(mat_translate_back, matrix)
    
    # Apply final translation to move the object to the final location
    matrix = mult_matrix(mat_translate, matrix)

    return matrix


In [23]:
# Generic function to draw a shape   
def draw_shape(start, end, step, mode, color):
    R, G, B = color
    glUniform4f(loc_color, R, G, B, 1.0)
    for triangle in range(start, end, step):
        glDrawArrays(mode, triangle, step)

def draw_rocket():
    # Draw the cone
    draw_shape(0, num_vertices_comp[0], 3, GL_TRIANGLES, (1.0, 0.0, 0.0))
    
    # Draw the cylinder
    draw_shape(num_vertices_comp[0], sum(num_vertices_comp[:2]), 3, GL_TRIANGLES, (0.8, 0.8, 0.8))

    # Draw prism1
    draw_shape(sum(num_vertices_comp[:2]), sum(num_vertices_comp[:3]) - 8, 3, GL_TRIANGLES, (1.0, 0.0, 0.0))
    draw_shape(sum(num_vertices_comp[:3]) - 8, sum(num_vertices_comp[:3]), 4, GL_TRIANGLE_STRIP, (1.0, 0.0, 0.0))

    # Draw prism2
    draw_shape(sum(num_vertices_comp[:3]), sum(num_vertices_comp[:4]) - 8, 3, GL_TRIANGLES, (1.0, 0.0, 0.0))
    draw_shape(sum(num_vertices_comp[:4]) - 8, sum(num_vertices_comp[:4]), 4, GL_TRIANGLE_STRIP, (1.0, 0.0, 0.0))

def draw_saturn():
    # Draw the sphere
    draw_shape(sum(num_vertices_comp[:4]), sum(num_vertices_comp[:5]), 3, GL_TRIANGLES, (0.9, 0.8, 0.5))

    # Draw the ring
    draw_shape(sum(num_vertices_comp[:5]), sum(num_vertices_comp[:6]), 3, GL_TRIANGLES, (0.8, 0.7, 0.6))

def draw_satellite():
    # Draw cylinder1
    draw_shape(sum(num_vertices_comp[:6]), sum(num_vertices_comp[:7]), 3, GL_TRIANGLES, (0.0, 0.0, 0.0))

    # Draw cone1
    draw_shape(sum(num_vertices_comp[:7]), sum(num_vertices_comp[:8]), 3, GL_TRIANGLES, (0.275, 0.275, 0.275))

    # Draw cone2
    draw_shape(sum(num_vertices_comp[:8]), sum(num_vertices_comp[:9]), 3, GL_TRIANGLES, (0.750, 0.750, 0.750))

    # Draw square1
    draw_shape(sum(num_vertices_comp[:9]), sum(num_vertices_comp[:10]), 4, GL_QUADS, (0.275, 0.275, 0.275))

    # Draw square2
    draw_shape(sum(num_vertices_comp[:10]), sum(num_vertices_comp[:11]), 4, GL_QUADS, (0.275, 0.275, 0.275))

    # Draw square3
    draw_shape(sum(num_vertices_comp[:11]), sum(num_vertices_comp[:12]), 4, GL_QUADS, (0.750, 0.750, 0.750))

    # Draw square4
    draw_shape(sum(num_vertices_comp[:12]), sum(num_vertices_comp[:13]), 4, GL_QUADS, (0.750, 0.750, 0.750))

def draw_star():
    draw_shape(sum(num_vertices_comp[:13]), sum(num_vertices_comp[:14]), 3, GL_TRIANGLES, (1.0, 1.0, 1.0))

def draw_moon():
    draw_shape(sum(num_vertices_comp[:14]), sum(num_vertices_comp[:15]), 3, GL_TRIANGLES, (0.9, 0.9, 0.1))

# Function that renders the objects
def render_objects():
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)    
    glClearColor(0.0, 0.071, 0.361, 1.0)
    for obj in objects:
        mat_transform = np.eye(4)
        mat_transform = apply_transformations(mat_transform, 
                                              translation=obj['transformation']['translation'],
                                              rotation=obj['transformation']['rotation'],
                                              scale=obj['transformation']['scale'],
                                              object_position=obj['transformation']['object_position'])
        glUniformMatrix4fv(loc_transformation, 1, GL_TRUE, mat_transform)
        obj['draw']()


In [24]:
glEnable(GL_DEPTH_TEST) 
loc_color = glGetUniformLocation(program, "color")
loc_transformation = glGetUniformLocation(program, "mat_transformation")

def main():
    while not glfw.window_should_close(window):
        glfw.poll_events() 
    
        render_objects()
    
        glfw.swap_buffers(window)
    
    glfw.terminate()

if __name__ == '__main__':
    main()

71
71
71
71
72
72
72
72
72
72
